### Loading the dataset

In [1]:
import re
import numpy as np


def cache (path,max_parents,n_nodes):
#with open('./data/scores/asia.jkl','r') as file:
    with open(path,'r') as file:
        filedata = file.readlines()
    
        
    #line = filedata[1]
    #print(line)
    #print(re.findall(r'\d+', line))

    n_parents_set=[]
    cache ={}
    
    
    
    t=1
    vertices=[]
    for i in range (n_nodes):
        line = filedata[t]
        vertices.append(int(re.findall(r'\d+', line)[0]))
        t+=int(re.findall(r'\d+', line)[1])+1
    #print(vertices)
    
    t=1
    for i in vertices :
        line = filedata[t]
        now_n_parents_set =int(re.findall(r'\d+', line)[1])
        #print(now_n_parents_set)
        n_parents_set.append(now_n_parents_set)
        now_dict= {}
        #print(filedata[t])
        for j in range(now_n_parents_set):
            now_parents=()
            for k in range (len(re.findall(r'\d+', filedata[t+j+1]))-3):
                #print(re.findall(r'\d+', filedata[t+j+1]))
                now_parents= now_parents + (int(re.findall(r'\d+', filedata[t+j+1])[k+3]),)
            #now_dict[float(filedata[t+j+1].split(' ')[0])]=now_parents
            now_parents=tuple(sorted(now_parents))
            #if now_parents==(12,18):
                #print(i)
            now_dict[now_parents]=float(filedata[t+j+1].split(' ')[0])
        cache[i]=now_dict
        t=t+now_n_parents_set+1
    return cache

cache=cache('data/scores/insurance-1000.jkl',3,27)
#n_parents_set



### Igraph REV functions

import numpy as np
import igraph as ig
from tqdm import tqdm
import random
import igraph as ig
import numpy as np
from scipy.special import binom
from src.utils import ScoreManager
from src.mcmc import mcmc as sample
from src.steps import REV

def propose_next(G_i: ig.Graph, is_REV, beta, score_manager: ScoreManager):
    new_edge_reversal_move = REV(score_manager, beta).new_edge_reversal_move

    return new_edge_reversal_move(G_i)

    
score_manager = ScoreManager('insurance50003p.jkl')  


### Temporary rev(to be removed later)

In [2]:
import numpy as np
import igraph as ig
from tqdm import tqdm
import random
import igraph as ig
import numpy as np
from scipy.special import binom
from src.utils import ScoreManager
from src.steps import REV

def propose_next(G_i: ig.Graph, is_REV, beta, score_manager: ScoreManager):
    new_edge_reversal_move = REV(score_manager, beta).new_edge_reversal_move

    return new_edge_reversal_move(G_i)

    
score_manager = ScoreManager('insurance-1000')  


### Functions for DAGs

In [3]:
import random

def is_dag(adjacency_matrix):
    num_nodes = adjacency_matrix.shape[0]

    # Color codes: 0 for not visited, 1 for visiting, 2 for visited
    colors = np.zeros(num_nodes, dtype=int)

    def dfs(node):
        nonlocal colors

        colors[node] = 1  # Mark the current node as visiting

        for neighbor in np.where(adjacency_matrix[node] == 1)[0]:
            if colors[neighbor] == 1:
                return False  # Found a backward edge (cycle)
            elif colors[neighbor] == 0:
                if not dfs(neighbor):
                    return False

        colors[node] = 2  # Mark the current node as visited
        return True

    # Start DFS from each unvisited node
    for node in range(num_nodes):
        if colors[node] == 0:
            if not dfs(node):
                return False  # Found a cycle

    return True  # No cycles found

def generate_random_dag_adjacency_matrix(num_nodes,max_parents):
    # Ensure a valid number of nodes
    if num_nodes <= 0:
        raise ValueError("Number of nodes must be greater than 0.")

    # Randomly order the nodes
    nodes = np.random.permutation(num_nodes)

    # Initialize an empty adjacency matrix
    adjacency_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    # Add edges to form a DAG
    for i in range(num_nodes - 1):
        # Randomly choose the number of edges from the current node
        num_edges = np.random.randint(1, num_nodes - i)

        # Randomly select nodes to connect
        target_nodes = np.random.choice(nodes[i+1:], size=num_edges, replace=False)

        # Add edges to the adjacency matrix
        adjacency_matrix[nodes[i], target_nodes] = 1

    new_dag= np.zeros((num_nodes,num_nodes),dtype=np.int8)
    for i in range (num_nodes):
        a=list(np.where(adjacency_matrix[:,i]==1)[0])
        b=random.randint(0,max_parents)
        c=random.sample(sorted(a), min(len(a),b))
        for j in c:
            new_dag[j,i]=1
    #is_dag(new_dag)
    #new_dag
    return new_dag


def dag_mean (x,start=0):
    dim=(len(x[1][:,0,0]))
    dag=np.zeros((len(x[1][:,0,0]),len(x[1][:,0,0])))
    t=int(len(x[2])-1)
    #print(t)
    for i in range(start,t):
        #print(i)
        dag += x[1][:,:,i]
    return dag/(len(x[2])-start) 


def score(dag):
    prior=1
    loglik=0

    for i in range(n_variables):
        prior= prior*(1/math.comb((n_variables-1),int(np.sum(dag[:,i]))))
        loglik+= cache[i][tuple(np.where(dag[:,i]==1)[0])]
    return loglik+ np.log(prior)


### 1 step of MCMCMC

The while cycle is useless now but It may be useful if I want to handle the constraints in a different way.

In [4]:
from math import comb

def mcmcmc (dag_now_p, loglik_now_p, prior_now_p, score_now_p, beta=1):
    global score_now, dag_now, loglik_now, prior_now,tot_swaps

    if heating==None:
        heating=1
    
    dag0=np.copy(dag_now_p)
    
    condition=False
    n_variables=len(dag_now_p[0])
    while(condition==False):
        u=random.randint(0, n_variables-1)
        v=list(range(n_variables))
        v.remove(u)
        v=random.choice(v)
        dag1=np.copy(dag_now_p)
        if (dag_now_p[u,v]==1) :
            dag1[u,v]=0
        elif (dag_now_p[v,u]==1):
            dag1[u,v]=1
            dag1[v,u]=0
            if (np.sum(dag_now_p[:,v])>=max_parents):
                dag1[u,v]=0
                dag1[v,u]=1
            if (is_dag(dag1)==False):
                dag1[u,v]=0
                dag1[v,u]=1
        elif (dag_now_p[u,v]==0 and dag_now_p[v,u]==0):
            dag1[u,v]=1
            if np.sum(dag_now_p[:,v])>=max_parents:
                dag1[u,v]=0
            if (is_dag(dag1)==False):
                dag1[u,v]=0
        condition=True

    prior_1=1
    loglik_1=0
    prior_0=1
    loglik_0=0

    for i in range(n_variables):
        prior_1= prior_1*(1/comb((n_variables-1),np.sum(dag1[:,i])))
        loglik_1+= cache[i][tuple(np.where(dag1[:,i]==1)[0])]
        prior_0= prior_0*(1/comb((n_variables-1),np.sum(dag0[:,i])))
        loglik_0+= cache[i][tuple(np.where(dag0[:,i]==1)[0])]
    prior_1=np.log(prior_1)
    prior_0=np.log(prior_0)
    
    #print(np.log(prior_1),loglik_1,np.log(prior_now_p),loglik_now_p)
    
    alpha= min(beta*(loglik_1-loglik_0)+prior_1-prior_0,0)
    #print(np.exp(alpha))
    #print((loglik_1-loglik_now_p),np.log(prior_1)-np.log(prior_now_p))
    #print(np.exp(alpha))
    #print(heating*(loglik_1-loglik_now_p)+np.log(prior_1)-np.log(prior_now_p))
    rand=random.random()
    #print(rand)


    if rand < np.exp(alpha):
        if (loglik_1-loglik_0)!=0:
            tot_swaps+=1
            #print((loglik_1-loglik_now_p),np.log(prior_1)-np.log(prior_now_p))
            #print(np.exp(alpha))
        #print('accepted')
        #print(dag_now-dag1)
        dag_now=np.copy(dag1)
        prior_now = np.copy(prior_1)
        loglik_now= np.copy(loglik_1)
        score_now= np.copy(loglik_1+prior_1)



### Main function basic structure MCMC

In [5]:
def structure_mcmc (n_variables, max_parents, n_iter, cache, prob_rev=0, seed=None):
    global score_now, dag_now, loglik_now, prior_now, tot_swaps

    tot_swaps=0
    random.seed(seed)
    np.random.seed(seed)

    dag_now= generate_random_dag_adjacency_matrix(n_variables,max_parents)

    dags =np.zeros((n_variables, n_variables, n_iter))
    scores=np.zeros(n_iter)

    prior_now=1
    loglik_now=0

    for i in range(n_variables):
        prior_now= prior_now*(1/comb((n_variables-1),np.sum(dag_now[:,i])))
        loglik_now+= cache[i][tuple(np.where(dag_now[:,i]==1)[0])]
    
    prior_now=np.log(prior_now)
    score_now= loglik_now+prior_now

    dags[:,:,0]=dag_now
    scores[0]=score_now

    for i in range(n_iter-1):
        if random.random() > prob_rev:
            mcmcmc(dag_now, loglik_now, prior_now, score_now)
            dags[:,:,i+1]=np.copy(dag_now)
            scores[i+1]=np.copy(score_now)
        else:
            G=ig.Graph.Adjacency(dag_now)
            G=propose_next(G, True, 1, score_manager)[0]
            dag_now=np.array(ig.Graph.get_adjacency(G).data)
            dags[:,:,i+1]=np.array(ig.Graph.get_adjacency(G).data)
            loglik_now, prior_now=score_manager.get_score(G)
            scores[i+1]= loglik_now+ prior_now

    print(tot_swaps,'swaps')

    return None,dags, scores



### Functions for parallel tempering

In [6]:
import scipy
import matplotlib.pyplot as plt
import splines

x=[1,2,3,4,5]
y=[1,4,4,11,19]
pchip=scipy.interpolate.PchipInterpolator(x, y, axis=0, extrapolate=None)

#X=np.arange(0,10,.1)
#plt.plot(X,pchip(X))
#plt.plot(x,y,'o')


def bisection_at_x (f,a,b,x,max_iter=100,tol=1e-6):
    for i in range (max_iter):
        c = (a + b) / 2
        if ((f(c) == x) or ((b - a) / 2 < tol)):
            return(c)
        if (f(c)< x):
            a = c
        else:
            b = c
    return (c)

x=np.array([1,2,3,4,5])

def y(x):
    return x**2

#bisection_at_x(y,1,5,3,100)**2

### DEO main function (parallel tempering)

1) t training steps are performed at the beginning to find the optimal number of chains and the values of the temperatures.

2)  outputs: \
    a) dags and scores of the posterior during the training steps(post_dags, post_scores) \
    b) dags and scores of the posterior after the training steps(final_post_dags, final_post_scores) \
    c) dags and scores of each chain after the training steps (final_dags, final_scores) \
    d) the indexes of the chains to keep track of the swaps (indexes_matrix, indexes_matrix_train)
    


In [7]:
import math
import scipy

def pt_structure_mcmc (n_variables_p, max_parents_p, n_iter, cache, training_iter, n_chains_p, prob_rev=.2, seed=None):
    global dags_now, loglik_now, prior_now, iterator, n_variables, max_parents, n_chains, steps, rev_post, rev_tot, rand_steps

    random.seed(seed)
    n_variables=int(n_variables_p)
    max_parents=int(max_parents_p)
    np.random.seed(seed)

    n_variables= n_variables_p
    n_chains=n_chains_p

    chain_rej=1/(n_chains-1)
    betas=np.zeros(n_chains)
    betas[n_chains-1]=1
    for i in range (n_chains-1):
        betas[i]=chain_rej*i

    training_steps= math.log2(training_iter)
    training_list= []
    training_counter=0
    for i in range (int(training_steps-2)):
        training_list.append(2**(i+1))
        training_counter+=2**(i+1)
    training_list.append(training_iter-training_counter)

    dags_now= np.zeros((n_variables,n_variables,n_chains),dtype=np.int8)
    loglik_now=np.zeros((n_chains))
    prior_now= np.zeros((n_chains))
    post_scores=  np.zeros(training_iter)
    post_dags= np.zeros((n_variables,n_variables,training_iter),dtype=np.int8)
    indexes_matrix_train=np.zeros((n_chains,training_iter))
    indexes_matrix_train[:,0]=np.array(list(range(n_chains)))


    for i in range(n_chains):
        dags_now[:,:,i]= generate_random_dag_adjacency_matrix(n_variables,max_parents)
        #print(is_dag( dags_now[:,:,i]))

    for j in range (n_chains):
        prior_t=1
        loglik_t=0
        for i in range(n_variables):
            #print(i)
            prior_t= prior_t*(1/math.comb((n_variables-1),int(np.sum(dags_now[:,i,j]))))
            loglik_t+= cache[i][tuple(np.where(dags_now[:,i,j]==1)[0])]
        prior_t=np.log(prior_t)
        prior_now[j]= prior_t
        loglik_now[j]= loglik_t

    steps=0
    counter=0
    rev_post=0
    rev_tot=0

    for i in training_list:
        r=np.zeros(n_chains)
        for k in range(i):
            for j in range (n_chains):
                iterator=j

                if random.random() > prob_rev:
                    pt_mcmcmc (dags_now, loglik_now, prior_now, betas[j])
                else:
                    G=ig.Graph.Adjacency(dags_now[:,:,j])
                    G=propose_next(G, True, betas[j], score_manager)[0]
                    dags_now[:,:,j]=np.array(ig.Graph.get_adjacency(G).data)
                    loglik_now[j], prior_now[j]=score_manager.get_score(G)
                    #print (loglik_now[j], prior_now[j])

            for j in range (n_chains-1):
                alpha= np.exp(min((loglik_now[j]-loglik_now[j+1])*(betas[j+1]-betas[j]),0))
                if alpha== np.nan:
                    alpha=0
                #print(alpha,j)
                r[j+1]+=(1-alpha)
                accepted= np.random.binomial(1, alpha, 1)
                #accepted=0
                if accepted==1 :
                    if (((j%2+k%2)==0) or ((j%2+k%2)==2)):
                        #print('swap',j)
                        ind1=np.copy(indexes_matrix_train[j,counter])
                        ind2=np.copy(indexes_matrix_train[j+1,counter])
                        indexes_matrix_train[j,counter]=ind2
                        indexes_matrix_train[j+1,counter]=ind1
                        d1=np.copy(dags_now[:,:,j])
                        d2=np.copy(dags_now[:,:,j+1])
                        dags_now[:,:,j+1]=d1
                        dags_now[:,:,j]=d2
                        l1=np.copy(loglik_now[j])
                        l2=np.copy(loglik_now[j+1])
                        loglik_now[j+1]=l1
                        loglik_now[j]=l2
                        p1=np.copy(prior_now[j])
                        p2=np.copy(prior_now[j+1])
                        prior_now[j+1]=p1
                        prior_now[j]=p2

            #print('after swap')
            #print(loglik_now)

            post_dags[:,:,counter]=dags_now[:,:,n_chains-1]
            post_scores[counter]=loglik_now[n_chains-1]+prior_now[n_chains-1]
            counter+=1
            if counter <training_iter:
                indexes_matrix_train[:,counter]=np.copy(indexes_matrix_train[:,counter-1])

            #print(dags_now[:,:,49]-dags_now[:,:,48])
        #print('post',loglik_now[-1]+prior_now[-1])
        r=r/(k+1)
        arr_lambda= np.cumsum(r)
        #print(arr_lambda)
        #print(betas)
        lambda_1=arr_lambda[-1]
        r_star=lambda_1/(n_chains-1)

        pchip=scipy.interpolate.PchipInterpolator(betas, arr_lambda, axis=0, extrapolate=None)
        X=np.arange(0,1,.001)
        plt.plot(X,pchip(X))
        plt.show()

        #print(betas,'betas_before')

        for j in range (n_chains):
            betas[j]=bisection_at_x(pchip,0,1,r_star*j)
        #print(betas)
        betas[0]=0
        betas[n_chains-1]=1
        #print(betas,'betas_after')

    print(steps,'steps')

    n_chains= int(2*lambda_1)
    r_star= lambda_1/(n_chains-1)
    betas= np.zeros(n_chains)
    for j in range (n_chains):
        betas[j]=bisection_at_x(pchip,0,1,r_star*j)
    betas[n_chains-1]=1
    betas[0]=0

    print('betas',betas)
    print('n_chains', n_chains)

    final_scores=  np.zeros((n_chains, n_iter))
    final_dags= np.zeros((n_variables,n_variables, n_chains, n_iter),dtype=np.int8)
    final_post_scores= np.zeros(n_iter)
    final_post_dags= np.zeros((n_variables, n_variables, n_iter),dtype=np.int8)
    indexes_matrix=np.zeros((n_chains,n_iter))
    indexes_matrix[:,0]=np.array(list(range(n_chains)))


    dags_now= dags_now[:,:,-n_chains:]
    loglik_now=loglik_now[-n_chains:]
    prior_now= prior_now[-n_chains:]

    #steps=0
    rev_post=0
    rev_tot=0

    counter=0
    r=np.zeros(n_chains)
    for k in range(n_iter):
        for j in range (n_chains):
            iterator=j
            if random.random() > prob_rev:
                pt_mcmcmc (dags_now, loglik_now, prior_now, betas[j])
            else:
                G=ig.Graph.Adjacency(dags_now[:,:,j])
                G=propose_next(G, True, betas[j], score_manager)[0]
                dags_now[:,:,j]=np.array(ig.Graph.get_adjacency(G).data)
                loglik_now[j], prior_now[j]=score_manager.get_score(G)
                #print (loglik_now[j], prior_now[j])
                #if prior_now[j]== np.nan:
                #   print('nan')

        
        #print ('before_swap',loglik_now, prior_now)
        for j in range (n_chains-1):
            alpha= np.exp(min((loglik_now[j]-loglik_now[j+1])*(betas[j+1]-betas[j]),0))
            if alpha== np.nan:
                alpha=0
            #print(alpha,j)
            r[j+1]+=(1-alpha)
            accepted= np.random.binomial(1, alpha, 1)
            #accepted=0
            
            if accepted==1 :
                #print('before swap', loglik_now)
                if (((j%2+k%2)==0) or ((j%2+k%2)==2)):
                    #print('swap',j)
                    ind1=np.copy(indexes_matrix[j,k])
                    ind2=np.copy(indexes_matrix[j+1,k])
                    indexes_matrix[j,k]=ind2
                    indexes_matrix[j+1,k]=ind1
                    d1=np.copy(dags_now[:,:,j])
                    d2=np.copy(dags_now[:,:,j+1])
                    dags_now[:,:,j+1]=d1
                    dags_now[:,:,j]=d2
                    l1=np.copy(loglik_now[j])
                    l2=np.copy(loglik_now[j+1])
                    loglik_now[j+1]=l1
                    loglik_now[j]=l2
                    p1=np.copy(prior_now[j])
                    p2=np.copy(prior_now[j+1])
                    prior_now[j+1]=p1
                    prior_now[j]=p2
                    #print('after swap', loglik_now)
        #print ('after_swap',loglik_now, prior_now)
        
        #print(loglik_now)

        for j in range (n_chains):
            #if prior_now[j]== np.nan:
                #print('nan')
            final_scores[j,k]=loglik_now[j]+prior_now[j]
            final_dags[:,:,j,k]=dags_now[:,:,j]
            #print(loglik_now[j]+np.log(prior_now[j]),loglik_now[j],np.log(prior_now[j]))
            if j==(n_chains-1):
                final_post_dags[:,:,k]=dags_now[:,:,j]
                final_post_scores[k]=loglik_now[j]+prior_now[j]
                #print('post',final_post_scores[k])
        if k <(n_iter-1):
                indexes_matrix[:,k+1]=np.copy(indexes_matrix[:,k])

        counter+=1

        if k%3000==0:
            r=r/(counter)
            arr_lambda= np.cumsum(r)
            #print(arr_lambda)
            #print(betas)
            lambda_1=arr_lambda[-1]
            r_star=lambda_1/(n_chains-1)

            print(betas,'betas')
            print(r,'r')
            print(loglik_now[-1]+prior_now[-1],'score post')

            pchip=scipy.interpolate.PchipInterpolator(betas, arr_lambda, axis=0, extrapolate=None)
            X=np.arange(0,1,.001)
            plt.plot(X,pchip(X))
            plt.show()

            #print(betas,'betas_before')

            for j in range (n_chains):
                betas[j]=bisection_at_x(pchip,0,1,r_star*j)
            #print(betas)
            betas[0]=0
            betas[n_chains-1]=1

            r=np.zeros(n_chains)
            counter=0



    #r=r/(k+1)
    #print('r',r)
    #arr_lambda= np.cumsum(r)
    #lambda_1=arr_lambda[-1]
    #r_star=lambda_1/(n_chains-1)

    #pchip=scipy.interpolate.PchipInterpolator(betas, arr_lambda, axis=0, extrapolate=None)
    #X=np.arange(0,1,.001)
    #plt.plot(X,pchip(X))
    #plt.show()

    print(steps,'steps_after')
    #print(rev_post,'rev_post')
    #print(rev_tot,'rev_tot')
    
    tot_post=np.concatenate([post_scores,final_post_scores])
    
    return tot_post, final_post_dags, final_post_scores, final_dags, final_scores, post_dags, post_scores, indexes_matrix, indexes_matrix_train


### SEO main function

In [8]:
import math
import scipy

def seo_structure_mcmc (n_variables_p, max_parents_p, n_iter, cache, training_iter, n_chains_p, prob_rev=.2, seed=None):
    global dags_now, loglik_now, prior_now, iterator, n_variables, max_parents, n_chains, steps, rev_post, rev_tot, rand_steps

    random.seed(seed)
    n_variables=int(n_variables_p)
    max_parents=int(max_parents_p)
    np.random.seed(seed)

    n_variables= n_variables_p
    n_chains=n_chains_p

    chain_rej=1/(n_chains-1)
    betas=np.zeros(n_chains)
    betas[n_chains-1]=1
    for i in range (n_chains-1):
        betas[i]=chain_rej*i

    training_steps= math.log2(training_iter)
    training_list= []
    training_counter=0
    for i in range (int(training_steps-2)):
        training_list.append(2**(i+1))
        training_counter+=2**(i+1)
    training_list.append(training_iter-training_counter)

    dags_now= np.zeros((n_variables,n_variables,n_chains),dtype=np.int8)
    loglik_now=np.zeros((n_chains))
    prior_now= np.zeros((n_chains))
    post_scores=  np.zeros(training_iter)
    post_dags= np.zeros((n_variables,n_variables,training_iter),dtype=np.int8)
    indexes_matrix_train=np.zeros((n_chains,training_iter))
    indexes_matrix_train[:,0]=np.array(list(range(n_chains)))


    for i in range(n_chains):
        dags_now[:,:,i]= generate_random_dag_adjacency_matrix(n_variables,max_parents)

    for j in range (n_chains):
        prior_t=1
        loglik_t=0
        for i in range(n_variables):
            prior_t= prior_t*(1/math.comb((n_variables-1),int(np.sum(dags_now[:,i,j]))))
            loglik_t+= cache[i][tuple(np.where(dags_now[:,i,j]==1)[0])]
        prior_t=np.log(prior_t)
        prior_now[j]= prior_t
        loglik_now[j]= loglik_t

    steps=0
    counter=0
    rev_post=0
    rev_tot=0

    for i in training_list:
        r=np.zeros(n_chains)
        for k in range(i):
            
            K=random.random()
            if K<.5:
                K=1
            else:
                K=2
                
            for j in range (n_chains):
                iterator=j

                if random.random() > prob_rev:
                    pt_mcmcmc (dags_now, loglik_now, prior_now, betas[j])
                else:
                    G=ig.Graph.Adjacency(dags_now[:,:,j])
                    G=propose_next(G, True, betas[j], score_manager)[0]
                    dags_now[:,:,j]=np.array(ig.Graph.get_adjacency(G).data)
                    loglik_now[j], prior_now[j]=score_manager.get_score(G)
                    #print (loglik_now[j], prior_now[j])

            for j in range (n_chains-1):
                alpha= np.exp(min((loglik_now[j]-loglik_now[j+1])*(betas[j+1]-betas[j]),0))
                if alpha== np.nan:
                    alpha=0
                #print(alpha,j)
                r[j+1]+=(1-alpha)
                accepted= np.random.binomial(1, alpha, 1)
                #accepted=0
                if accepted==1 :
                    if (((j%2+K%2)==0) or ((j%2+K%2)==2)):
                        #print('swap',j)
                        ind1=np.copy(indexes_matrix_train[j,counter])
                        ind2=np.copy(indexes_matrix_train[j+1,counter])
                        indexes_matrix_train[j,counter]=ind2
                        indexes_matrix_train[j+1,counter]=ind1
                        d1=np.copy(dags_now[:,:,j])
                        d2=np.copy(dags_now[:,:,j+1])
                        dags_now[:,:,j+1]=d1
                        dags_now[:,:,j]=d2
                        l1=np.copy(loglik_now[j])
                        l2=np.copy(loglik_now[j+1])
                        loglik_now[j+1]=l1
                        loglik_now[j]=l2
                        p1=np.copy(prior_now[j])
                        p2=np.copy(prior_now[j+1])
                        prior_now[j+1]=p1
                        prior_now[j]=p2

            #print('after swap')
            #print(loglik_now)

            post_dags[:,:,counter]=dags_now[:,:,n_chains-1]
            post_scores[counter]=loglik_now[n_chains-1]+prior_now[n_chains-1]
            counter+=1
            if counter <training_iter:
                indexes_matrix_train[:,counter]=np.copy(indexes_matrix_train[:,counter-1])

            #print(dags_now[:,:,49]-dags_now[:,:,48])
        #print('post',loglik_now[-1]+prior_now[-1])
        r=r/(k+1)
        arr_lambda= np.cumsum(r)
        #print(arr_lambda)
        #print(betas)
        lambda_1=arr_lambda[-1]
        r_star=lambda_1/(n_chains-1)

        pchip=scipy.interpolate.PchipInterpolator(betas, arr_lambda, axis=0, extrapolate=None)
        X=np.arange(0,1,.001)
        plt.plot(X,pchip(X))
        plt.show()

        #print(betas,'betas_before')

        for j in range (n_chains):
            betas[j]=bisection_at_x(pchip,0,1,r_star*j)
        #print(betas)
        betas[0]=0
        betas[n_chains-1]=1
        #print(betas,'betas_after')

    print(steps,'steps')

    n_chains= int(2*lambda_1)
    r_star= lambda_1/(n_chains-1)
    betas= np.zeros(n_chains)
    for j in range (n_chains):
        betas[j]=bisection_at_x(pchip,0,1,r_star*j)
    betas[n_chains-1]=1
    betas[0]=0

    print('betas',betas)
    print('n_chains', n_chains)

    final_scores=  np.zeros((n_chains, n_iter))
    final_dags= np.zeros((n_variables,n_variables, n_chains, n_iter),dtype=np.int8)
    final_post_scores= np.zeros(n_iter)
    final_post_dags= np.zeros((n_variables, n_variables, n_iter),dtype=np.int8)
    indexes_matrix=np.zeros((n_chains,n_iter))
    indexes_matrix[:,0]=np.array(list(range(n_chains)))


    dags_now= dags_now[:,:,-n_chains:]
    loglik_now=loglik_now[-n_chains:]
    prior_now= prior_now[-n_chains:]

    #steps=0
    rev_post=0
    rev_tot=0

    counter=0
    r=np.zeros(n_chains)
    for k in range(n_iter):
        K=random.random()
        if K<.5:
            K=1
        else:
            K=2
        
        for j in range (n_chains):
            iterator=j
            if random.random() > prob_rev:
                pt_mcmcmc (dags_now, loglik_now, prior_now, betas[j])
            else:
                G=ig.Graph.Adjacency(dags_now[:,:,j])
                G=propose_next(G, True, betas[j], score_manager)[0]
                dags_now[:,:,j]=np.array(ig.Graph.get_adjacency(G).data)
                loglik_now[j], prior_now[j]=score_manager.get_score(G)
                #print (loglik_now[j], prior_now[j])
                #if prior_now[j]== np.nan:
                #   print('nan')

        
        #print ('before_swap',loglik_now, prior_now)
        for j in range (n_chains-1):
            alpha= np.exp(min((loglik_now[j]-loglik_now[j+1])*(betas[j+1]-betas[j]),0))
            if alpha== np.nan:
                alpha=0
            #print(alpha,j)
            r[j+1]+=(1-alpha)
            accepted= np.random.binomial(1, alpha, 1)
            #accepted=0
            
            if accepted==1 :
                #print('before swap', loglik_now)
                if (((j%2+K%2)==0) or ((j%2+K%2)==2)):
                    #print('swap',j)
                    ind1=np.copy(indexes_matrix[j,k])
                    ind2=np.copy(indexes_matrix[j+1,k])
                    indexes_matrix[j,k]=ind2
                    indexes_matrix[j+1,k]=ind1
                    d1=np.copy(dags_now[:,:,j])
                    d2=np.copy(dags_now[:,:,j+1])
                    dags_now[:,:,j+1]=d1
                    dags_now[:,:,j]=d2
                    l1=np.copy(loglik_now[j])
                    l2=np.copy(loglik_now[j+1])
                    loglik_now[j+1]=l1
                    loglik_now[j]=l2
                    p1=np.copy(prior_now[j])
                    p2=np.copy(prior_now[j+1])
                    prior_now[j+1]=p1
                    prior_now[j]=p2
                    #print('after swap', loglik_now)
        #print ('after_swap',loglik_now, prior_now)
        
        #print(loglik_now)

        for j in range (n_chains):
            #if prior_now[j]== np.nan:
                #print('nan')
            final_scores[j,k]=loglik_now[j]+prior_now[j]
            final_dags[:,:,j,k]=dags_now[:,:,j]
            #print(loglik_now[j]+np.log(prior_now[j]),loglik_now[j],np.log(prior_now[j]))
            if j==(n_chains-1):
                final_post_dags[:,:,k]=dags_now[:,:,j]
                final_post_scores[k]=loglik_now[j]+prior_now[j]
                #print('post',final_post_scores[k])
        if k <(n_iter-1):
                indexes_matrix[:,k+1]=np.copy(indexes_matrix[:,k])

        counter+=1

        if k%3000==0:
            r=r/(counter)
            arr_lambda= np.cumsum(r)
            #print(arr_lambda)
            #print(betas)
            lambda_1=arr_lambda[-1]
            r_star=lambda_1/(n_chains-1)

            print(betas,'betas')
            print(r,'r')
            print(loglik_now[-1]+prior_now[-1],'score post')

            pchip=scipy.interpolate.PchipInterpolator(betas, arr_lambda, axis=0, extrapolate=None)
            X=np.arange(0,1,.001)
            plt.plot(X,pchip(X))
            plt.show()

            #print(betas,'betas_before')

            for j in range (n_chains):
                betas[j]=bisection_at_x(pchip,0,1,r_star*j)
            #print(betas)
            betas[0]=0
            betas[n_chains-1]=1

            r=np.zeros(n_chains)
            counter=0



    #r=r/(k+1)
    #print('r',r)
    #arr_lambda= np.cumsum(r)
    #lambda_1=arr_lambda[-1]
    #r_star=lambda_1/(n_chains-1)

    #pchip=scipy.interpolate.PchipInterpolator(betas, arr_lambda, axis=0, extrapolate=None)
    #X=np.arange(0,1,.001)
    #plt.plot(X,pchip(X))
    #plt.show()

    print(steps,'steps_after')
    #print(rev_post,'rev_post')
    #print(rev_tot,'rev_tot')
    
    tot_post=np.concatenate([post_scores,final_post_scores])
    
    return tot_post, final_post_dags, final_post_scores, final_dags, final_scores, post_dags, post_scores, indexes_matrix, indexes_matrix_train


### Random swap PT main function

In [9]:
import math
import scipy

def rand_pt_structure_mcmc (n_variables_p, max_parents_p, n_iter, cache, training_iter, n_chains_p, prob_rev=.2, seed=None):
    global dags_now, loglik_now, prior_now, iterator, n_variables, max_parents, n_chains, steps, rev_post, rev_tot, rand_steps

    random.seed(seed)
    n_variables=int(n_variables_p)
    max_parents=int(max_parents_p)
    np.random.seed(seed)

    n_variables= n_variables_p
    n_chains=n_chains_p

    chain_rej=1/(n_chains-1)
    betas=np.zeros(n_chains)
    betas[n_chains-1]=1
    for i in range (n_chains-1):
        betas[i]=chain_rej*i

    training_steps= math.log2(training_iter)
    training_list= []
    training_counter=0
    for i in range (int(training_steps-2)):
        training_list.append(2**(i+1))
        training_counter+=2**(i+1)
    training_list.append(training_iter-training_counter)

    dags_now= np.zeros((n_variables,n_variables,n_chains),dtype=np.int8)
    loglik_now=np.zeros((n_chains))
    prior_now= np.zeros((n_chains))
    post_scores=  np.zeros(training_iter)
    post_dags= np.zeros((n_variables,n_variables,training_iter),dtype=np.int8)
    indexes_matrix_train=np.zeros((n_chains,training_iter))
    indexes_matrix_train[:,0]=np.array(list(range(n_chains)))


    for i in range(n_chains):
        dags_now[:,:,i]= generate_random_dag_adjacency_matrix(n_variables,max_parents)

    for j in range (n_chains):
        prior_t=1
        loglik_t=0
        for i in range(n_variables):
            prior_t= prior_t*(1/math.comb((n_variables-1),int(np.sum(dags_now[:,i,j]))))
            loglik_t+= cache[i][tuple(np.where(dags_now[:,i,j]==1)[0])]
        prior_t=np.log(prior_t)
        prior_now[j]= prior_t
        loglik_now[j]= loglik_t

    steps=0
    counter=0
    rev_post=0
    rev_tot=0

    for i in training_list:
        r=np.zeros(n_chains)
        for k in range(i):
            
            for j in range (n_chains):
                iterator=j

                if random.random() > prob_rev:
                    pt_mcmcmc (dags_now, loglik_now, prior_now, betas[j])
                else:
                    G=ig.Graph.Adjacency(dags_now[:,:,j])
                    G=propose_next(G, True, betas[j], score_manager)[0]
                    dags_now[:,:,j]=np.array(ig.Graph.get_adjacency(G).data)
                    loglik_now[j], prior_now[j]=score_manager.get_score(G)
                    #print (loglik_now[j], prior_now[j])

            for j in range (n_chains-1):
                alpha= np.exp(min((loglik_now[j]-loglik_now[j+1])*(betas[j+1]-betas[j]),0))
                if alpha== np.nan:
                    alpha=0
                #print(alpha,j)
                r[j+1]+=(1-alpha)
                accepted= np.random.binomial(1, alpha, 1)
                #accepted=0
        
                if accepted==1 :
                    K=random.random()
                    if (K<.5):
                        #print('swap',j)
                        ind1=np.copy(indexes_matrix_train[j,counter])
                        ind2=np.copy(indexes_matrix_train[j+1,counter])
                        indexes_matrix_train[j,counter]=ind2
                        indexes_matrix_train[j+1,counter]=ind1
                        d1=np.copy(dags_now[:,:,j])
                        d2=np.copy(dags_now[:,:,j+1])
                        dags_now[:,:,j+1]=d1
                        dags_now[:,:,j]=d2
                        l1=np.copy(loglik_now[j])
                        l2=np.copy(loglik_now[j+1])
                        loglik_now[j+1]=l1
                        loglik_now[j]=l2
                        p1=np.copy(prior_now[j])
                        p2=np.copy(prior_now[j+1])
                        prior_now[j+1]=p1
                        prior_now[j]=p2

            #print('after swap')
            #print(loglik_now)

            post_dags[:,:,counter]=dags_now[:,:,n_chains-1]
            post_scores[counter]=loglik_now[n_chains-1]+prior_now[n_chains-1]
            counter+=1
            if counter <training_iter:
                indexes_matrix_train[:,counter]=np.copy(indexes_matrix_train[:,counter-1])

            #print(dags_now[:,:,49]-dags_now[:,:,48])
        #print('post',loglik_now[-1]+prior_now[-1])
        r=r/(k+1)
        arr_lambda= np.cumsum(r)
        #print(arr_lambda)
        #print(betas)
        lambda_1=arr_lambda[-1]
        r_star=lambda_1/(n_chains-1)

        pchip=scipy.interpolate.PchipInterpolator(betas, arr_lambda, axis=0, extrapolate=None)
        X=np.arange(0,1,.001)
        plt.plot(X,pchip(X))
        plt.show()

        #print(betas,'betas_before')

        for j in range (n_chains):
            betas[j]=bisection_at_x(pchip,0,1,r_star*j)
        #print(betas)
        betas[0]=0
        betas[n_chains-1]=1
        #print(betas,'betas_after')

    print(steps,'steps')

    n_chains= int(2*lambda_1)
    r_star= lambda_1/(n_chains-1)
    betas= np.zeros(n_chains)
    for j in range (n_chains):
        betas[j]=bisection_at_x(pchip,0,1,r_star*j)
    betas[n_chains-1]=1
    betas[0]=0

    print('betas',betas)
    print('n_chains', n_chains)

    final_scores=  np.zeros((n_chains, n_iter))
    final_dags= np.zeros((n_variables,n_variables, n_chains, n_iter),dtype=np.int8)
    final_post_scores= np.zeros(n_iter)
    final_post_dags= np.zeros((n_variables, n_variables, n_iter),dtype=np.int8)
    indexes_matrix=np.zeros((n_chains,n_iter))
    indexes_matrix[:,0]=np.array(list(range(n_chains)))


    dags_now= dags_now[:,:,-n_chains:]
    loglik_now=loglik_now[-n_chains:]
    prior_now= prior_now[-n_chains:]

    #steps=0
    rev_post=0
    rev_tot=0

    counter=0
    r=np.zeros(n_chains)
    for k in range(n_iter):
        
        for j in range (n_chains):
            iterator=j
            if random.random() > prob_rev:
                pt_mcmcmc (dags_now, loglik_now, prior_now, betas[j])
            else:
                G=ig.Graph.Adjacency(dags_now[:,:,j])
                G=propose_next(G, True, betas[j], score_manager)[0]
                dags_now[:,:,j]=np.array(ig.Graph.get_adjacency(G).data)
                loglik_now[j], prior_now[j]=score_manager.get_score(G)
                #print (loglik_now[j], prior_now[j])
                #if prior_now[j]== np.nan:
                #   print('nan')

        
        #print ('before_swap',loglik_now, prior_now)
        for j in range (n_chains-1):
            alpha= np.exp(min((loglik_now[j]-loglik_now[j+1])*(betas[j+1]-betas[j]),0))
            if alpha== np.nan:
                alpha=0
            #print(alpha,j)
            r[j+1]+=(1-alpha)
            accepted= np.random.binomial(1, alpha, 1)
            #accepted=0
            
            if accepted==1 :
                K=random.random()
                #print('before swap', loglik_now)
                if (K<.5):
                    #print('swap',j)
                    ind1=np.copy(indexes_matrix[j,k])
                    ind2=np.copy(indexes_matrix[j+1,k])
                    indexes_matrix[j,k]=ind2
                    indexes_matrix[j+1,k]=ind1
                    d1=np.copy(dags_now[:,:,j])
                    d2=np.copy(dags_now[:,:,j+1])
                    dags_now[:,:,j+1]=d1
                    dags_now[:,:,j]=d2
                    l1=np.copy(loglik_now[j])
                    l2=np.copy(loglik_now[j+1])
                    loglik_now[j+1]=l1
                    loglik_now[j]=l2
                    p1=np.copy(prior_now[j])
                    p2=np.copy(prior_now[j+1])
                    prior_now[j+1]=p1
                    prior_now[j]=p2
                    #print('after swap', loglik_now)
        #print ('after_swap',loglik_now, prior_now)
        
        #print(loglik_now)

        for j in range (n_chains):
            #if prior_now[j]== np.nan:
                #print('nan')
            final_scores[j,k]=loglik_now[j]+prior_now[j]
            final_dags[:,:,j,k]=dags_now[:,:,j]
            #print(loglik_now[j]+np.log(prior_now[j]),loglik_now[j],np.log(prior_now[j]))
            if j==(n_chains-1):
                final_post_dags[:,:,k]=dags_now[:,:,j]
                final_post_scores[k]=loglik_now[j]+prior_now[j]
                #print('post',final_post_scores[k])
        if k <(n_iter-1):
                indexes_matrix[:,k+1]=np.copy(indexes_matrix[:,k])

        counter+=1

        if k%3000==0:
            r=r/(counter)
            arr_lambda= np.cumsum(r)
            #print(arr_lambda)
            #print(betas)
            lambda_1=arr_lambda[-1]
            r_star=lambda_1/(n_chains-1)

            print(betas,'betas')
            print(r,'r')
            print(loglik_now[-1]+prior_now[-1],'score post')

            pchip=scipy.interpolate.PchipInterpolator(betas, arr_lambda, axis=0, extrapolate=None)
            X=np.arange(0,1,.001)
            plt.plot(X,pchip(X))
            plt.show()

            #print(betas,'betas_before')

            for j in range (n_chains):
                betas[j]=bisection_at_x(pchip,0,1,r_star*j)
            #print(betas)
            betas[0]=0
            betas[n_chains-1]=1

            r=np.zeros(n_chains)
            counter=0



    #r=r/(k+1)
    #print('r',r)
    #arr_lambda= np.cumsum(r)
    #lambda_1=arr_lambda[-1]
    #r_star=lambda_1/(n_chains-1)

    #pchip=scipy.interpolate.PchipInterpolator(betas, arr_lambda, axis=0, extrapolate=None)
    #X=np.arange(0,1,.001)
    #plt.plot(X,pchip(X))
    #plt.show()

    print(steps,'steps_after')
    #print(rev_post,'rev_post')
    #print(rev_tot,'rev_tot')
    
    tot_post=np.concatenate([post_scores,final_post_scores])
    
    return tot_post, final_post_dags, final_post_scores, final_dags, final_scores, post_dags, post_scores, indexes_matrix, indexes_matrix_train


### Random step parallel temprering

In [10]:
def pt_random (dags_now_p, loglik_now_p, prior_now_p, beta):
    global dags_now, loglik_now, prior_now, iterator, n_variables, max_parents, n_chains, steps, rev_post, rev_tot, rand_steps

    dag0=np.copy(dags_now_p[:,:,iterator])
    
    dag1= generate_random_dag_adjacency_matrix(n_variables,max_parents)

    prior_1=1
    loglik_1=0

    prior_0=1
    loglik_0=0

    for i in range(n_variables):
        prior_1= prior_1*(1/math.comb((n_variables-1),int(np.sum(dag1[:,i]))))
        loglik_1+= cache[i][tuple(np.where(dag1[:,i]==1)[0])]
        prior_0= prior_0*(1/math.comb((n_variables-1),int(np.sum(dag0[:,i]))))
        loglik_0+= cache[i][tuple(np.where(dag0[:,i]==1)[0])]

    alpha= min(beta*(loglik_1-loglik_0)+np.log(prior_1)-np.log(prior_0),0)
    
    rand=random.random()
    
    print(np.exp(alpha),iterator)
    
    if rand < np.exp(alpha):
        if (np.sum(np.abs(dags_now_p[:,:,iterator]-dag1))!=0) :
            steps+=1

        dags_now[:,:,iterator]=np.copy(dag1)
        prior_now[iterator] = prior_1
        loglik_now[iterator]= loglik_1
        #print(dags_now)

### MCMCMC step parallel tempering

In [11]:
def pt_mcmcmc (dags_now_p, loglik_now_p, prior_now_p, beta):
    global dags_now, loglik_now, prior_now, iterator, n_variables, max_parents, n_chains, steps, rev_post, rev_tot, rand_steps

    condition=False
    while(condition==False):
        u=random.randint(0, n_variables-1)
        v=list(range(n_variables))
        v.remove(u)
        v=random.choice(v)
        dag1=np.copy(dags_now_p[:,:,iterator])
        if (dags_now_p[u,v,iterator]==1) :
            dag1[u,v]=0

        elif (dags_now_p[v,u,iterator]==1):
            dag1[u,v]=1
            dag1[v,u]=0
            if  (np.sum(dags_now_p[:,v,iterator])>=max_parents):
                dag1[u,v]=0
                dag1[v,u]=1
                #continue
            if (is_dag(dag1)==False):
                dag1[u,v]=0
                dag1[v,u]=1
        elif (dags_now_p[u,v,iterator]==0 and dags_now_p[v,u,iterator]==0):
            dag1[u,v]=1
            if  np.sum(dags_now_p[:,v,iterator])>=max_parents:
                dag1[u,v]=0
                #continue
            if (is_dag(dag1)==False):
                dag1[u,v]=0
        condition=True

    dag0=np.copy(dags_now_p[:,:,iterator])
    prior_1=1
    loglik_1=0

    prior_0=1
    loglik_0=0

    for i in range(n_variables):
        prior_1= prior_1*(1/math.comb((n_variables-1),int(np.sum(dag1[:,i]))))
        loglik_1+= cache[i][tuple(np.where(dag1[:,i]==1)[0])]
        prior_0= prior_0*(1/math.comb((n_variables-1),int(np.sum(dag0[:,i]))))
        loglik_0+= cache[i][tuple(np.where(dag0[:,i]==1)[0])]
        
    prior_0=np.log(prior_0)
    prior_1=np.log(prior_1)
    ###print(np.log(prior_1),loglik_1,np.log(prior_0),loglik_0)
    #if iterator==n_chains -1:
        #print ('post')
        #print(dag1-dags_now_p[:,:,iterator])
    #print(iterator,dags_now_p[:,:,iterator])
    alpha= min(beta*(loglik_1-loglik_0)+prior_1-prior_0,0)
    #print(beta*(loglik_1-loglik_now_p[iterator]),'lik')
    #print(np.log(prior_1)-np.log(prior_now_p[iterator]),'prior')
    #print(loglik_1,loglik_now[iterator])
    ###print(np.exp(alpha),'alpha',iterator)
    #print(np.exp(alpha))
    #print(heating*(loglik_1-loglik_now_p)+np.log(prior_1)-np.log(prior_now_p))
    rand=random.random()
    #print(rand)
    #print(np.log(prior_1),loglik_1,np.log(prior_now_p[iterator]),loglik_now_p[iterator])
    #print(np.sum(np.abs(dags_now_p[:,:,iterator]-dag1)),np.exp(alpha),'alpha',iterator)
    if rand < np.exp(alpha):
        #print('accepted')
        #print(dags_now[:,:,iterator]-dag1)
        #print('mc3',iterator)
        #if np.sum(dag1 -dags_now_p[:,:,iterator])!=0:
        #    print(np.log(prior_1),loglik_1,np.log(prior_now_p[iterator]),loglik_now_p[iterator])
        #   print(np.exp(alpha),'alpha',iterator)
        #print(dags_now_p[:,:,iterator]-dag1)
        if (np.sum(np.abs(dags_now_p[:,:,iterator]-dag1))!=0) :
            steps+=1

        dags_now[:,:,iterator]=np.copy(dag1)
        prior_now[iterator] = prior_1
        loglik_now[iterator]= loglik_1
        #print(dags_now)

### REV step parallel tempering

In [12]:
def pt_rev (dags_now_p, loglik_now_p, prior_now_p, beta):
    global dags_now, loglik_now, prior_now, iterator, n_variables, max_parents, n_chains, steps, rev_post, rev_tot,rand_steps

    condition= False
    dag1=np.copy(dags_now_p[:,:,iterator])
    dag0=dags_now_p[:,:,iterator]
    #print(dag0)

    if np.sum(dag0)==0:
        return None

    while(condition==False):
        ind=np.where(dag0==1)
        indexes=[]
        #print(ind,ind[0][1])
        for i in range (len(ind[0])):
            indexes.append([ind[0][i],ind[1][i]])

        edge= random.sample(indexes,1)[0]
        u=edge[0]
        v=edge[1]
        #print(u,v)

        pi_j=np.where(dag0[:,v])

        dag1[:,u]=np.zeros(n_variables)
        dag1[:,v]=np.zeros(n_variables)

        #print(dag1)

        d_u=np.where(dag1[u,:]==1)[0]
        dd_u=list(np.copy(d_u))
        #print(dd_u)
        visited=[]
        while len(dd_u)!=0:
            child=dd_u.pop()
            #print(child,'child')
            if child not in visited:
                visited.append(child)
                for nephew in np.where(dag1[child,:]==1)[0]:
                    #print(nephew,'nephew')
                    d_u=np.append(d_u,nephew)
                    dd_u.append(nephew)
                #print(dd_u)
                dd_u=list(set(dd_u))
                #print(dd_u)
        d_u=set(d_u)
        #print(d_u)

        d_v=np.where(dag1[v,:]==1)[0]
        dd_v=list(np.copy(d_v))
        #print(dd_v)
        visited=[]
        while len(dd_v)!=0:
            child=dd_v.pop()
            #rint(child,'child')
            if child not in visited:
                visited.append(child)
                for nephew in np.where(dag1[child,:]==1)[0]:
                    #print(nephew,'nephew')
                    d_v=np.append(d_v,nephew)
                    dd_v.append(nephew)
                #print(dd_u)
                dd_v=list(set(dd_v))
                #print(dd_v)
        d_v=set(d_v)
        #print(d_v)

        pi_i_set=[item for item in cache[u] if v in item]
        for i in d_u:
            pi_i_set=[item for item in pi_i_set if i not in item]

        pi_i_tilde=random.sample(list(cache[u].keys()),1)[0]
        if pi_i_tilde not in pi_i_set:
            dag1=np.copy(dag0)
            break
        for i in pi_i_tilde:
            dag1[i,u]=1

        d_v_plus=np.where(dag1[v,:]==1)[0]
        dd_v=list(np.copy(d_v_plus))
        #print(dd_v)
        visited=[]
        while len(dd_v)!=0:
            child=dd_v.pop()
            #print(child,'child')
            if child not in visited:
                visited.append(child)
                for nephew in np.where(dag1[child,:]==1)[0]:
                    #print(nephew,'nephew')
                    d_v_plus=np.append(d_v_plus,nephew)
                    dd_v.append(nephew)
                #print(dd_u)
                dd_v=list(set(dd_v))
                #print(dd_v)
        d_v_plus=set(d_v_plus)
        #print(d_v_plus)

        pi_j_set=[item for item in cache[v]]
        for i in d_v_plus:
            pi_j_set=[item for item in pi_j_set if i not in item]

        pi_j_tilde=random.sample(list(cache[v].keys()),1)[0]
        if pi_j_tilde not in pi_j_set:
            dag1=np.copy(dag0)
            break
        for i in pi_j_tilde:
            dag1[i,v]=1

        condition= True

    prior_1=1
    loglik_1=0

    prior_0=1
    loglik_0=0

    for i in range(n_variables):
        prior_1= prior_1*(1/math.comb((n_variables-1),int(np.sum(dag1[:,i]))))
        loglik_1+= cache[i][tuple(np.where(dag1[:,i]==1)[0])]
        prior_0= prior_0*(1/math.comb((n_variables-1),int(np.sum(dag0[:,i]))))
        loglik_0+= cache[i][tuple(np.where(dag0[:,i]==1)[0])]
    #print(np.log(prior_1),loglik_1,np.log(prior_now_p),loglik_now_p)

    alpha= min(beta*(loglik_1-loglik_0)+np.log(prior_1)-np.log(prior_0),0)
    #print(np.exp(alpha),iterator)
    #print(beta*(loglik_1-loglik_now_p[iterator]))
    rand=random.random()
    #print(rand)
    if rand < np.exp(alpha):
        #print('rev',iterator)
        #print('accepted')
        #print(dag_now-dag1)
        dags_now[:,:,iterator]=np.copy(dag1)
        prior_now[iterator] = prior_1
        loglik_now[iterator]= loglik_1

        if np.sum(np.abs(dag0-dag1))!=0:
            rev_tot+=1
            if iterator== (n_chains-1):
                rev_post+=1